In [ ]:
from og_marl.vault_utils.download_vault import *
from og_marl.vault_utils.analyse_vault import *
import os
import json
from IPython.display import display, HTML
import shutil

In [ ]:
def make_one_card(source,env,task,datacard_dict):
    info_1_card = datacard_dict[source][env][task]

    # make the relevant details table
    table_components = ["Environment name","Version","Agents","Action size","Observation size","Reward type"]

    version_dict = {
        "og_marl":{
            "smac_v1": "<a href='https://github.com/oxwhirl/smac/releases/tag/v1'>SMAC V1</a>, from OxWhiRL",
            "smac_v2": "<a href='https://github.com/oxwhirl/smacv2'>SMAC V2</a>, from OxWhiRL",
            "mamujoco": "<a href='https://github.com/schroederdewitt/multiagent_mujoco/releases/tag/v1.1'>V1.1</a>, Mujoco v210",
        },
        "cfcql":{
            "smac_v1": "<a href='https://github.com/oxwhirl/smac/releases/tag/v1'>SMAC V1</a>, from OxWhiRL",
        },
        "alberdice":{
            "rware": "<a href='https://github.com/dematsunaga/alberdice/tree/main/marl_env/marl/env'>Code included in Alberdice repository</a>",
        },
        "omar":{
            "mpe":"<a href='https://github.com/ling-pan/OMAR/tree/master/multiagent-particle-envs'>Code included in OMAR repository<a>",
            "mamujoco": "<a href='https://github.com/schroederdewitt/multiagent_mujoco/releases/tag/v1.0'>V1.0</a>, Mujoco v200",
        },
        "omiga": {
            "smac_v1":"Modified version of SMAC v1, popularised by <a href='https://github.com/sanmuyang/multi-agent-PPO-on-SMAC?tab=readme-ov-file'>MAPPO </a>",
            "mamujoco": "<a href='https://github.com/schroederdewitt/multiagent_mujoco/releases/tag/v1.0'>V1.0</a>, Mujoco v200",
        }
    }

    uid = list(info_1_card.keys())[0]

    action_size = 1
    if len(info_1_card[uid]['Structure']['actions'])>3:
        action_size = info_1_card[uid]['Structure']['actions'][2:]

    tab_1 = "<table><tr>"
    for comp in table_components:
        tab_1+=f"<th>{comp}</th>"
    tab_1+="</tr><tr>"
    tab_1+=f"<td>{info_1_card[uid]['Environment name']}</td>"
    tab_1+=f"<td>{version_dict[source][env]}</td>"
    tab_1+=f"<td>{info_1_card[uid]['Structure']['actions'][2]}</td>"
    tab_1+=f"<td>{action_size}</td>"
    tab_1+=f"<td>{info_1_card[uid]['Structure']['observations'][3:]}</td>"
    tab_1+=f"<td>Dense</td>"
    tab_1+="</tr></table>"

    # Make the summary statistics table
    table_components = ["Min return","Max return","Transitions","Trajectories","Joint SACo"]

    html_table = "<table><tr>"
    html_table+="<th>Uid</th>"
    html_table+=f"<th>Episode return mean</th>"
    for comp in table_components:
        html_table+=f"<th>{comp}</th>"

    for uid in info_1_card.keys():
        html_table+="</tr><tr>"
        html_table+=f"<td>{uid}</td>"
        html_table+=str(fr"<td>{info_1_card[uid]['Mean episode return']:.2f} ")+'&#177;'+ str(f" {info_1_card[uid]['Standard deviation episode return']:.2f}</td>")
        for comp in table_components[:2]:
            html_table+=f"<td>{info_1_card[uid][comp]:.2f}</td>"
        for comp in table_components[2:-1]:
            html_table+=f"<td>{info_1_card[uid][comp]}</td>"
        for comp in table_components[-1:]:
            html_table+=f"<td>{info_1_card[uid][comp]:.2f}</td>"
    html_table+="</tr></table>"

    # image url
    # image_url = info_1_card[uid]['Histogram download url']
    # image_url = "/home/louise/workspace/og-marl/docs/assets/vault_plots/og_marl/smac_v1/5m_vs_6m_histogram.pdf"
    image_url = f"../assets/vault_plots/{source}/{env}/{task}_histogram.png"
    # print(image_url)
    # image_url = 'blob:null/a6506be2-d5d5-4ba5-a39f-dfe2cdc618b6'

    one_datacard = f"<div class=\"card\"><img src=\"{image_url}\" alt=\"{info_1_card[uid]['Scenario name']}\" class=\"card-img\"><div class=\"card-content\"><h2>{info_1_card[uid]['Scenario name']} - <a href='{info_1_card[uid]['Download link']}'>Download</a></h2><h3>Metadata</h3><p>{tab_1}</p><h3>Generation procedure for each dataset</h3><p>{info_1_card[uid]['Generation procedure']}</p><h3>Summary statistics</h3><p>{html_table}</p></div></div>"

    # display(HTML(one_datacard))
    print(one_datacard)
    
    return one_datacard

In [ ]:

with open("./datacard_info.json") as current_info:
    # get json string
    datacard_str = json.load(current_info)

    # convert to dictionary
    datacard_dict = json.loads(datacard_str)

In [ ]:
# source = "og_marl"

pretty_source_names = {
    "og_marl": "OG MARL",
    "cfcql": "CFCQL",
    "omiga": "OMIGA",
    "omar":"OMAR",
    "alberdice":"Alberdice",
}

with open("./datacard_info.json") as current_info:
    # get json string
    datacard_str = json.load(current_info)

    # convert to dictionary
    datacard_dict = json.loads(datacard_str)

for source in datacard_dict.keys():
    start = f"<!DOCTYPE html><html lang=\"en\"><head>    <meta charset=\"UTF-8\">    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">    <title>Dataset Cards - {pretty_source_names[source]}</title>    <link rel=\"stylesheet\" href=\"styles.css\">    <style>    * "
    start += "{    margin: 0;    padding: 0;    box-sizing: border-box;} body {    font-family: Arial, sans-serif;    background-color: #f4f4f4;} .container {    max-width: 1200px;    margin: 0 auto;    padding: 20px;} h1 {    text-align: center;    margin-bottom: 20px;} .card-grid {    display: grid;    grid-template-columns: repeat(auto-fill, minmax(500px, 1fr));    gap: 20px;} .card {    background-color: white;    border-radius: 8px;    box-shadow: 0 2px 5px rgba(0,0,0,0.1);    overflow: hidden;} .card-img {    width: 100%;    height: auto;} .card-content {    padding: 15px;} .card-content h2 {    font-size: 1.5em;    margin-bottom: 10px;} .card-content p {    color: #555;}    </style></head><body>    <div class=\"container\">        "
    start+= f"<h1>Dataset Cards - {pretty_source_names[source]}</h1>        <div class=\"card-grid\">"

    for env in datacard_dict[source].keys():
        if not env=="gymnasium_mamujoco":
            for task in datacard_dict[source][env].keys():
                start += make_one_card(source,env,task,datacard_dict)

    start += "</div></div></body></html>"

    display(HTML(start))

    with open(f'{source}.md', 'w') as f:
        f.write(start)
